# Importations

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv("data/ai4i2020.csv")
df

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


In [28]:
drop_columns = ["UDI", "Product ID", "TWF", "HDF", "PWF", "OSF", "RNF"]
utils_columns = ["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]"]

# Pre-processing

### TrainTest - Nettoyage - Encodage

In [24]:
trainset, testset = train_test_split(df, test_size=0.2, random_state=0)

In [25]:
trainset["Machine failure"].value_counts()

Machine failure
0    7736
1     264
Name: count, dtype: int64

In [26]:
testset["Machine failure"].value_counts()

Machine failure
0    1925
1      75
Name: count, dtype: int64

### Features Engineering

In [27]:
def features_engineering(df):
    df['power_machine'] = df['Torque [Nm]'] * (  3.14 * df['Rotational speed [rpm]'] / 30 )
    df['temp_diff'] = df['Process temperature [K]'] - df['Air temperature [K]']
    return df

In [ ]:
def imputation(df):
    df = df.drop([drop_columns, utils_columns])
    return df

In [31]:
def preprocessing(df):
    df = features_engineering(df)
    df = imputation(df)

    X, y = df.drop("Machine failure"),  df["Machine failure"]
    print(y.value_counts())
    
    return X, y
    

In [ ]:
X_train, y_train = preprocessing(trainset)
X_test, y_test = preprocessing(testset)

Models

Evaluation

Optimisation